In [1]:
import numpy as np

lfp_data=np.load('lfp_signal.npy')



In [ ]:
from rainbow_mouse.SIREN.lib import visualize_embedding_3d_multifreq_siren

#visualize_embedding_3d_multifreq_siren(lfp_data[:3000], "/home/maria/RainbowMouse/src/rainbow_mouse/SIREN/cebra_multifreq_siren.pt", 95, 100, 3, num_layers=3)

ModuleNotFoundError: No module named 'tkinter'

In [3]:
import matplotlib.pyplot as plt
import torch
import numpy as np
from rainbow_mouse.SIREN.lib import MultiFreqSIRENLayer, MultiFreqSIRENEncoder

def visualize_frequency_profile(model, data, frequencies, layer_index=0, title="Frequency Sensitivity"):
    """
    Computes and plots average activation magnitude per frequency in a MultiFreqSIREN layer.
    """
    model.eval()
    with torch.no_grad():
        x = torch.tensor(data, dtype=torch.float32)
        hidden = x

        # Walk through layers until the one we want
        for i, layer in enumerate(model.network):
            hidden = layer(hidden)
            if isinstance(layer, MultiFreqSIRENLayer) and i == layer_index:
                break

        # hidden now has shape [B, D * num_frequencies]
        num_frequencies = len(frequencies)
        units = hidden.shape[1] // num_frequencies

        # Reshape to [B, units, frequencies]
        hidden = hidden.view(hidden.shape[0], units, num_frequencies)

        # Compute mean absolute activation across batch
        mean_activation = hidden.abs().mean(dim=0)  # shape [units, frequencies]
        mean_activation_per_freq = mean_activation.mean(dim=0).cpu().numpy()

    # Plot
    plt.figure(figsize=(8, 4))
    plt.bar(range(len(frequencies)), mean_activation_per_freq, tick_label=[f"{f:.1f}" for f in frequencies])
    plt.title(title)
    plt.xlabel("Frequency (ω)")
    plt.ylabel("Mean Activation (|sin(ω * z)|)")
    plt.tight_layout()
    plt.show()

input_dim=95
hidden_dim=100
output_dim=3
encoder_path='/home/maria/RainbowMouse/src/rainbow_mouse/SIREN/cebra_multifreq_siren.pt'
encoder = MultiFreqSIRENEncoder(input_dim, hidden_dim, output_dim)
encoder.load_state_dict(torch.load(encoder_path))
encoder.eval()
visualize_frequency_profile(
    model=encoder,
    data=lfp_data[:2000],           # or a subset of interest
    frequencies=[5, 10, 30, 60],    # must match your model
    layer_index=0,                  # which SIREN layer to analyze
    title="Frequency Sensitivity in Layer 0"
)


/tmp/ipykernel_98314/3984163131.py:39: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [8]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from rainbow_mouse.SIREN.lib import LearnableFreqSIRENEncoder
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA

def animate_embedding_3d_learnable_siren(data, encoder_path, input_dim, hidden_dim, output_dim,
                                         num_layers=3, num_frequencies=6,
                                         omega_init_range=(1.0, 60.0),
                                         title="3D Embedding Animation (Learnable ω)",
                                         save_path="embedding_rotation_learnable.mp4"):
    # Load trained LearnableFreqSIRENEncoder with correct init config
    encoder = LearnableFreqSIRENEncoder(
        input_dim=input_dim,
        hidden_dim=hidden_dim,
        output_dim=output_dim,
        num_layers=num_layers,
        num_frequencies=num_frequencies,
        omega_init_range=omega_init_range
    )
    encoder.load_state_dict(torch.load(encoder_path))
    encoder.eval()

    # Encode the data
    with torch.no_grad():
        embeddings = encoder(torch.tensor(data, dtype=torch.float32))

    # Reduce to 3D if needed
    if output_dim > 3:
        embeddings = PCA(n_components=3).fit_transform(embeddings.numpy())
    else:
        embeddings = embeddings.numpy()

    # Create figure
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_title(title)
    ax.set_xlabel("Latent 1")
    ax.set_ylabel("Latent 2")
    ax.set_zlabel("Latent 3")

    # Plot initial points
    scatter = ax.plot(embeddings[:, 0], embeddings[:, 1], embeddings[:, 2], '.', color='indigo')[0]

    # Update function for rotation
    def update(angle):
        ax.view_init(elev=30, azim=angle)
        return fig,

    ani = animation.FuncAnimation(fig, update, frames=np.arange(0, 360, 2), blit=False)

    # Save the animation
    if save_path.endswith(".gif"):
        ani.save(save_path, writer="pillow", fps=20)
    else:
        ani.save(save_path, writer="ffmpeg", fps=20)

    print(f"Saved rotating embedding animation to {save_path}")
animate_embedding_3d_learnable_siren(
    data=lfp_data[:3000],
    encoder_path="cebra_learnable_siren.pt",
    input_dim=95,
    hidden_dim=100,
    output_dim=3,
    num_layers=3,
    num_frequencies=6,
    omega_init_range=(1.0, 60.0),
    save_path="embedding_rotation_learnable.mp4"
)

Saved rotating embedding animation to embedding_rotation_learnable.mp4


In [10]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA

def animate_embedding_3d_learnable_siren(data, encoder_path, input_dim, hidden_dim, output_dim,
                                         num_layers=3, num_frequencies=6,
                                         omega_init_range=(1.0, 60.0),
                                         title="3D Embedding Animation (Learnable ω)",
                                         save_path="embedding_rotation_learnable.mp4"):
    # Load trained model
    encoder = LearnableFreqSIRENEncoder(
        input_dim=input_dim,
        hidden_dim=hidden_dim,
        output_dim=output_dim,
        num_layers=num_layers,
        num_frequencies=num_frequencies,
        omega_init_range=omega_init_range
    )
    encoder.load_state_dict(torch.load(encoder_path))
    encoder.eval()

    # Get embeddings
    with torch.no_grad():
        embeddings = encoder(torch.tensor(data, dtype=torch.float32))

    if output_dim > 3:
        embeddings = PCA(n_components=3).fit_transform(embeddings.numpy())
    else:
        embeddings = embeddings.numpy()

    # Setup figure and style
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_title(title, fontsize=14)
    ax.set_xlabel("Latent 1", fontsize=12)
    ax.set_ylabel("Latent 2", fontsize=12)
    ax.set_zlabel("Latent 3", fontsize=12)

    # Plot static points (style-matched)
    scatter = ax.plot(embeddings[:, 0], embeddings[:, 1], embeddings[:, 2], color='seagreen')[0]

    # Rotation update function
    def update(angle):
        ax.view_init(elev=30, azim=angle)
        return fig,

    # Animate
    ani = animation.FuncAnimation(fig, update, frames=np.arange(0, 360, 2), blit=False)

    # Save
    if save_path.endswith(".gif"):
        ani.save(save_path, writer="pillow", fps=20)
    else:
        ani.save(save_path, writer="ffmpeg", fps=20)

    print(f"Saved rotating embedding animation to {save_path}")
animate_embedding_3d_learnable_siren(
    data=lfp_data[:3000],
    encoder_path="cebra_learnable_siren.pt",
    input_dim=95,
    hidden_dim=100,
    output_dim=3,
    num_layers=3,
    num_frequencies=6,
    omega_init_range=(1.0, 60.0),
    save_path="embedding_rotation_learnable2.mp4"
)

Saved rotating embedding animation to embedding_rotation_learnable2.mp4
